In [2]:
# Desinstala a versão errada e instala a correta (3.5.1)
%pip uninstall pyspark -y
%pip install pyspark==3.5.1 --user --upgrade --force-reinstall

Found existing installation: pyspark 3.5.0
Can't uninstall 'pyspark'. No files were found to uninstall.
Note: you may need to restart the kernel to use updated packages.
  Using cached pyspark-3.5.1-py2.py3-none-any.whl
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 6.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, first
import pyspark

# 1. Verificar a versão instalada (TEM QUE SER 3.5.1)
print(f"Versão do PySpark no Jupyter: {pyspark.__version__}")

# 1. Iniciando a Sessão Spark
# SparkSession: É o ponto de entrada. Sem ele, nada acontece.
# .master(...): Diz quem manda. 'spark://spark-master:7077' é o endereço do container mestre.
# .config(...defaultFS...): Ensina o Spark que, quando eu falar "/pasta", é no HDFS, não no local.
# .config(...hive...): Habilita suporte a salvar tabelas SQL.
spark = SparkSession.builder \
    .appName("ETL_Silver") \
    .master("spark://spark-master:7077") \
    .config("spark.hadoop.fs.defaultFS", "hdfs://namenode:9000") \
    .config("spark.sql.warehouse.dir", "/user/hive/warehouse") \
    .enableHiveSupport() \
    .getOrCreate()

data = [("Java", 1), ("Python", 2)]
df = spark.createDataFrame(data, ["Linguagem", "ID"])

print("Se você ver a tabela abaixo, o problema foi resolvido:")
df.show()

# Função para cada url
def processar_silver(nome_arquivo, prefixo):
    print(f"Processando {nome_arquivo}...")
    
    # 2. Leitura (Read)
    # .read.json(): O Spark lê o arquivo e infere o esquema (descobre o que é texto, o que é numero).
    df = spark.read.json(f"/datalake/bronze/{nome_arquivo}.json")
    
    # 3. Seleção e Renomeação (Select/Alias)
    # col("NomeVelho").alias("NomeNovo"): Pega a coluna e dá um apelido amigável.
    # .cast("double"): Força o valor numérico a ser Decimal (para não virar texto por engano).
    df_limpo = df.select(
        col("SpatialDim").alias("Pais"),
        col("TimeDim").alias("Ano"),
        col("Dim1").alias("Sexo_Codigo"),
        col("NumericValue").cast("double").alias("Valor")
    )
    
    # 4. Lógica de Negócio
    # groupBy: Agrupa os dados para não duplicar País/Ano.
    # pivot("Sexo_Codigo"): Pega os valores dessa coluna (BTSX, FMLE, MLE) e transforma em NOVAS COLUNAS.
    # agg(first("Valor")): Como preencher essas novas colunas? Pegue o primeiro valor que achar.
    df_pivot = df_limpo.groupBy("Pais", "Ano") \
        .pivot("Sexo_Codigo") \
        .agg(first("Valor"))
        
    # 5. Renomear as colunas geradas pelo Pivot
    # withColumnRenamed: Troca o nome técnico da API por algo legível.
    df_final = df_pivot \
        .withColumnRenamed("SEX_BTSX", f"{prefixo}_Ambos") \
        .withColumnRenamed("SEX_FMLE", f"{prefixo}_Fem") \
        .withColumnRenamed("SEX_MLE",  f"{prefixo_coluna}_Masc")
    
    # 6. Escrita (Write)
    # .write.mode("overwrite"): Sobrescreve se já existir.
    # .parquet(): Formato colunar comprimido.
    caminho_destino = f"/datalake/silver/{nome_arquivo}"
    df_final.write.mode("overwrite").parquet(caminho_destino)
    print(f"✅ Salvo em Parquet: {caminho_destino}")

# Rodando
#processar_silver("raw_suicidio", "Suicidio")
#processar_silver("raw_depressao", "Depressao")
spark.stop()
print("Sessão Spark finalizada.")

Versão do PySpark no Jupyter: 3.5.0
Se você ver a tabela abaixo, o problema foi resolvido:


Py4JJavaError: An error occurred while calling o49.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 0.0 failed 4 times, most recent failure: Lost task 0.3 in stage 0.0 (TID 3) (172.27.0.5 executor 0): java.io.InvalidClassException: org.apache.spark.rdd.RDD; local class incompatible: stream classdesc serialVersionUID = 823754013007382808, local class serialVersionUID = 3516924559342767982
	at java.base/java.io.ObjectStreamClass.initNonProxy(Unknown Source)
	at java.base/java.io.ObjectInputStream.readNonProxyDesc(Unknown Source)
	at java.base/java.io.ObjectInputStream.readClassDesc(Unknown Source)
	at java.base/java.io.ObjectInputStream.readNonProxyDesc(Unknown Source)
	at java.base/java.io.ObjectInputStream.readClassDesc(Unknown Source)
	at java.base/java.io.ObjectInputStream.readOrdinaryObject(Unknown Source)
	at java.base/java.io.ObjectInputStream.readObject0(Unknown Source)
	at java.base/java.io.ObjectInputStream.defaultReadFields(Unkno